[View in Colaboratory](https://colab.research.google.com/github/nishanthjois/machine-learning/blob/master/Netflix.ipynb)

In [0]:
# this is just to know how much time will it take to run this entire ipython notebook 
from datetime import datetime
# globalstart = datetime.now()
import pandas as pd
import numpy as np
import matplotlib
#matplotlib.use('nbagg')

import matplotlib.pyplot as plt
plt.rcParams.update({'figure.max_open_warning': 0})

import seaborn as sns
sns.set_style('whitegrid')
import os
from scipy import sparse
from scipy.sparse import csr_matrix

from sklearn.decomposition import TruncatedSVD
from sklearn.metrics.pairwise import cosine_similarity
import random

In [0]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
··········
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
Please enter the verification code: Access token retrieved correctly.


In [0]:
!mkdir -p drive
!google-drive-ocamlfuse drive


In [0]:
!ls drive/data

data_small.csv	test1.csv  train1.csv


In [0]:

# print("creating the dataframe from data.csv file..")
# df = pd.read_csv('drive/data/data_small.csv', sep=',', 
#                        names=['movie', 'user','rating','date'])
# df.date = pd.to_datetime(df.date)
# print('Done.\n')

# # we are arranging the ratings according to time.
# print('Sorting the dataframe by date..')
# df.sort_values(by='date', inplace=True)
# print('Done..')

creating the dataframe from data.csv file..
Done.

Sorting the dataframe by date..
Done..


In [0]:
#df.describe()

,movie,user,rating
count,2.405376e+07,2.405376e+07,2.405376e+07
mean,2.308324e+03,1.322285e+06,3.599634e+00
std,1.303909e+03,7.645779e+05,1.086118e+00
min,1.000000e+00,6.000000e+00,1.000000e+00
25%,1.180000e+03,6.609270e+05,3.000000e+00
50%,2.342000e+03,1.318602e+06,4.000000e+00
75%,3.433000e+03,1.984358e+06,4.000000e+00
max,4.499000e+03,2.649429e+06,5.000000e+00


In [0]:
# # just to make sure that all Nan containing rows are deleted..
# print("No of Nan values in our dataframe : ", sum(df.isnull().any()))

No of Nan values in our dataframe :  0


In [0]:
# # Remove duplicates
# dup_bool = df.duplicated(['movie','user','rating'])
# dups = sum(dup_bool) # by considering all columns..( including timestamp)
# print("There are {} duplicate rating entries in the data..".format(dups))

There are 0 duplicate rating entries in the data..


In [0]:
# # Split train and test data
# if not os.path.isfile('train.csv'):
#     # create the dataframe and store it in the disk for offline purposes..
#     df.iloc[:int(df.shape[0]*0.80)].to_csv("train.csv", index=False)

# if not os.path.isfile('test.csv'):
#     # create the dataframe and store it in the disk for offline purposes..
#     df.iloc[int(df.shape[0]*0.80):].to_csv("test.csv", index=False)

# train_df = pd.read_csv("train.csv", parse_dates=['date'])
# test_df = pd.read_csv("test.csv")

*** Start from here if you a sample train1.csv and test1.csv palced in drive, else if you have only data.csv then start from top***

In [0]:
## Start from here if you a sample train1.csv and test1.csvdrive

train_df = pd.read_csv('drive/data/train1.csv')
test_df = pd.read_csv('drive/data/test1.csv')


In [6]:
print("Training data ")
print("-"*50)
print("\nTotal no of ratings :",train_df1.shape[0])
print("Total No of Users   :", len(np.unique(train_df.user)))
print("Total No of movies  :", len(np.unique(train_df.movie)))

Training data 
--------------------------------------------------

Total no of ratings : 19243011
Total No of Users   : 389223
Total No of movies  : 4403


In [7]:
print("Test data ")
print("-"*50)
print("\nTotal no of ratings :",test_df.shape[0])
print("Total No of Users   :", len(np.unique(test_df.user)))
print("Total No of movies  :", len(np.unique(test_df.movie)))

Test data 
--------------------------------------------------

Total no of ratings : 4810753
Total No of Users   : 302727
Total No of movies  : 4497


In [12]:
train_df.tail()

,movie,user,rating,date
19243006,1221,2162676,1,2005-08-01
19243007,1221,1637761,2,2005-08-01
19243008,2955,1082163,4,2005-08-01
19243009,2032,2457620,3,2005-08-01
19243010,2032,1687117,2,2005-08-01
